In [1]:
import matplotlib as mpl
mpl.use("pgf")
import numpy as np
mm_to_in = 1/25.4
pgf_with_pdflatex = {
    "pgf.texsystem": "pdflatex",
    "pgf.preamble": [
         r"\usepackage{mathtools}",
         ],
    "text.usetex": True,
    "figure.figsize": [90*mm_to_in, 3.25],
    "axes.labelsize": 10,
    "axes.labelpad": 5.0,
    "font.size": 12,
    "font.family": "serif",
    "legend.fontsize": 8,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "lines.linewidth": 1.5,
    "xtick.major.size": 5,
    "xtick.major.width": 1,
    "xtick.minor.size": 2.5,
    "xtick.minor.width": 1,
    "ytick.major.size": 5,
    "ytick.major.width": 1,
    "ytick.minor.size": 2.5,
    "ytick.minor.width": 1,
}
mpl.rcParams.update(pgf_with_pdflatex)

from scipy.optimize import curve_fit
from scipy import __version__
print(__version__)
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, AutoMinorLocator

0.19.0


In [2]:
def antoine(T, A, B, C):
    return A - B/(T - C)
ortega_data = np.loadtxt('vapor_pressure/ortega.csv', delimiter=',')
vanGenderen_data = np.loadtxt('vapor_pressure/vanGenderen.csv', delimiter=',')
verevkin_data = np.loadtxt('vapor_pressure/verevkin.csv', delimiter=',')
all_data = np.vstack((ortega_data, vanGenderen_data, verevkin_data))
all_data = all_data[all_data[:, 0].argsort()]
coeffs, cov_mat = curve_fit(antoine, all_data[:, 0], np.log10(all_data[:, 1]), p0=[6, 1500, 50], maxfev=10000)
print(coeffs)
conf_int = 2*np.sqrt(np.diag(cov_mat))
print(conf_int)
ortega_fit, _ = curve_fit(antoine, ortega_data[:, 0], np.log10(ortega_data[:, 1]), p0=[6, 1500, 50])
print(ortega_fit)

[    6.40297589  1528.68913524    52.88138973]
[  0.09186087  53.47122789   4.93379075]
[    6.21051367  1415.38995238    63.91337448]


In [3]:
ortega = np.array([6.23175, 1429.00, 62.30])
camacho = np.array([5.9644, 1281.06, 75.94])
stephenson = np.array([6.62646, 1658.4, 42.09])

In [4]:
temps = np.arange(np.floor(all_data[0, 0]), np.ceil(all_data[-1, 0]), 1)

In [20]:
fig, ax = plt.subplots()
ax.set_yscale('log')
ax.set_xlabel('$T$, K', labelpad=5.0)
ax.set_ylabel(r'$P_{\text{sat}}$, kPa', labelpad=2.0)
ax.set_ylim(0.5E-1, 2.0E2)
ax.set_xlim(270, 430)
ax.xaxis.set_major_locator(MaxNLocator(6))
ax.xaxis.set_minor_locator(AutoMinorLocator(3))
# ax.plot(temps, 10**antoine(temps, *ortega), label=r"Ortega et al.\ \cite{Ortega2003}")
ortega, = ax.plot(ortega_data[:, 0], ortega_data[:, 1], marker='o', linestyle='None', label=r"Ortega et al.\ \cite{Ortega2003}", markersize=3)
# ax.plot(temps, 10**antoine(temps, *camacho), label=r"Camacho et al.\ \cite{Camacho2007}")
# ax.plot(temps, 10**antoine(temps, *stephenson), label=r"Stephenson et al.\ \cite{Stephenson1987}")
vanGen, = ax.plot(vanGenderen_data[:, 0], vanGenderen_data[:, 1], marker='x', linestyle='None', label=r"van Genderen et al.\ \cite{vanGenderen2002}")
verevkin, = ax.plot(verevkin_data[:, 0], verevkin_data[:, 1], marker='+', linestyle='None', label=r"Verevkin and Emel'yanenko \cite{Verevkin2008}")
fit_line, = ax.plot(temps, 10**antoine(temps, *coeffs), label="This Work")
ax.fill_between(temps, 10**antoine(temps, *(coeffs+conf_int)), 10**antoine(temps, *(coeffs-conf_int)), alpha=0.5, facecolor='C2',)
fit_fill = ax.fill(np.NaN, np.NaN, facecolor='C2', alpha=0.5)
# handles, labels = ax.get_legend_handles_labels()
ax.legend([ortega, vanGen, verevkin, (fit_line, fit_fill)], ['Ortega', 'van Gen', 'Verevkin', 'This Work'], loc='best');
# ax.legend(loc='best');

AttributeError: 'NoneType' object has no attribute 'create_artists'

In [18]:
fig.tight_layout()
fig.savefig('vapor_pressure.pdf')